 # 🔹 **UNet con ReLu, BatchNorm y Upsampling Bilineal**

Este código define un **modelo UNet** para segmentación de imágenes.
Se enfoca en mejorar la estabilidad y el entrenamiento usando **Batch Normalization** y **upsampling bilineal**.

### 1️⃣ **DoubleConv**
- Dos capas convolucionales 3x3.
- **BatchNorm2d** para normalizar cada batch y mejorar la convergencia.
- Activación **ReLU**.
- Entrada: `in_ch` canales.
- Salida: `out_ch` canales.

### 2️⃣ **Encoder (Downsampling)**
- 4 niveles de convoluciones con **MaxPooling 2x2**.
- Cada nivel aumenta el número de filtros:
  - `64 → 128 → 256 → 512`.
- Nivel inferior (bottom) con 1024 filtros.

### 3️⃣ **Decoder (Upsampling)**
- Se usa **F.interpolate** (upsampling bilineal) para aumentar la resolución.
- Concatenación con los mapas de características del encoder (**skip connections**).
- Los filtros se reducen progresivamente:
  - `1024+512 → 512`, `512+256 → 256`, `256+128 → 128`, `128+64 → 64`.

### 4️⃣ **Salida**
- Convolución final 1x1 a `out_channels`.
- Activación **sigmoid**, adecuada para segmentación binaria.

### 5️⃣ **Uso**
```python
model = UNet(in_channels=3, out_channels=1)
output = model(input_tensor)  # input_tensor: [batch, 3, H, W]


In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super().__init__()
        self.down1 = DoubleConv(in_channels, 64)
        self.down2 = DoubleConv(64, 128)
        self.down3 = DoubleConv(128, 256)
        self.down4 = DoubleConv(256, 512)
        self.bottom = DoubleConv(512, 1024)
        self.up4 = DoubleConv(1024+512, 512)
        self.up3 = DoubleConv(512+256, 256)
        self.up2 = DoubleConv(256+128, 128)
        self.up1 = DoubleConv(128+64, 64)
        self.maxpool = nn.MaxPool2d(2)
        self.out = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        c1 = self.down1(x)
        c2 = self.down2(self.maxpool(c1))
        c3 = self.down3(self.maxpool(c2))
        c4 = self.down4(self.maxpool(c3))
        b = self.bottom(self.maxpool(c4))

        u4 = F.interpolate(b, size=c4.shape[2:], mode='bilinear', align_corners=True)
        u4 = self.up4(torch.cat([u4, c4], dim=1))
        u3 = F.interpolate(u4, size=c3.shape[2:], mode='bilinear', align_corners=True)
        u3 = self.up3(torch.cat([u3, c3], dim=1))
        u2 = F.interpolate(u3, size=c2.shape[2:], mode='bilinear', align_corners=True)
        u2 = self.up2(torch.cat([u2, c2], dim=1))
        u1 = F.interpolate(u2, size=c1.shape[2:], mode='bilinear', align_corners=True)
        u1 = self.up1(torch.cat([u1, c1], dim=1))
        return torch.sigmoid(self.out(u1))